##### Configurações Iniciais

In [17]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import joblib

from Modules.operation_analiser import OperationLoader, OperationAnalyzer
from Modules.forecast_loader import ForecastLoader
from Modules.variables_info import variables_info

In [ ]:
# Parâmetros gerais
lat_porto = -25.500511
lon_porto = -48.519331
forecast_path = '../Database/forecast_data'
paradas_path = '../Database/Parada_2024_chuva.xlsx'
timebox = ('202405', '202412')

variaveis = variables_info['variaveis']
legendas = variables_info['legendas']
legendas_dict = dict(zip(variaveis, legendas))

# Repositorios de Saída
base_output = '../Output/Analise_03'
os.makedirs('../Output/Analise_03', exist_ok=True)
for subfolder in ['Modelos', 'Tables', 'Imagens', 'Imagens/Metricas', 'Imagens/Modelos']:
    os.makedirs(os.path.join(base_output, subfolder), exist_ok=True)

##### Importação dos Dados para Analise

In [19]:
forecast = ForecastLoader(forecast_path, lat_target=lat_porto, lon_target=lon_porto)
df_forecast = forecast.load_forecasts(timebox, drop_duplicates=True)

operation_loader = OperationLoader(paradas_path)
df_paradas = operation_loader.load()

analyzer = OperationAnalyzer(df_paradas, df_forecast)
df_unificado = analyzer.marcar_paradas()

In [20]:
melhores = pd.read_csv(f'../Output/Analise_02/Tables/melhores_thresholds.csv')

#### Treinamento do Modelo Supervisionado para Classificação das Recomendações

In [21]:
# Define os cenários com base nas métricas
cenarios = {
    'conservador': melhores.sort_values('recall', ascending=False).head(3),
    'moderado': melhores.sort_values('f1_score', ascending=False).head(3),
    'arrojado': melhores.sort_values('precision', ascending=False).head(3),
}

In [ ]:
metricas_modelos = []

for nome_cenario, df_regras in cenarios.items():
    print(f"\n=== CENÁRIO: {nome_cenario.upper()} ===")

    # Gera variáveis binárias com base nos thresholds
    for _, row in df_regras.iterrows():
        col = row['variavel']
        thresh = row['threshold']
        df_unificado[f"{col}_bin"] = (df_unificado[col] > thresh).astype(int)

    variaveis_modelo = [f"{v}_bin" for v in df_regras['variavel'].unique().tolist()]
    X = df_unificado[variaveis_modelo]
    y = df_unificado['parada']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, stratify=y, random_state=42
    )

    # Modelos
    model_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model_rf.fit(X_train, y_train)

    model_lr = LogisticRegression(max_iter=1000, class_weight='balanced')
    model_lr.fit(X_train, y_train)

    # Avaliação Random Forest
    y_pred_rf = model_rf.predict(X_test)
    print("Random Forest")
    print(classification_report(y_test, y_pred_rf))
    print("Matriz de Confusão: ", confusion_matrix(y_test, y_pred_rf))

    report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
    for classe in ['0', '1']:
        for metric in ['precision', 'recall', 'f1-score']:
            metricas_modelos.append({
                'cenario': nome_cenario,
                'modelo': 'RandomForest',
                'classe': classe,
                'metric': metric,
                'value': report_rf[classe][metric]
            })

    # Avaliação Regressão Logística
    y_pred_lr = model_lr.predict(X_test)
    print("Logistic Regression")
    print(classification_report(y_test, y_pred_lr))
    print("Matriz de Confusão: ", confusion_matrix(y_test, y_pred_lr))

    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    for classe in ['0', '1']:
        for metric in ['precision', 'recall', 'f1-score']:
            metricas_modelos.append({
                'cenario': nome_cenario,
                'modelo': 'LogisticRegression',
                'classe': classe,
                'metric': metric,
                'value': report_lr[classe][metric]
            })

    # Salva modelos
    modelo_rf_path = f"{base_output}/Modelos/model_randomforest_{nome_cenario}.pkl"
    modelo_lr_path = f"{base_output}/Modelos/model_logisticregression_{nome_cenario}.pkl"
    joblib.dump(model_rf, modelo_rf_path)
    joblib.dump(model_lr, modelo_lr_path)
    print(f"Modelos salvos: {modelo_rf_path} e {modelo_lr_path}")

    variaveis_originais = [v.replace('_bin', '') for v in variaveis_modelo]
    df_vars = pd.DataFrame({
        'variavel': variaveis_originais,
        'variavel_bin': variaveis_modelo
    })
    for modelo_nome in ['logisticregression', 'randomforest']:
        var_path = f"{base_output}/Modelos/variaveis_{modelo_nome}_{nome_cenario}.csv"
        df_vars.to_csv(var_path, index=False, header=True)

    # Exporta métricas (de todos os cenários juntos)
    metricas_df = pd.DataFrame(metricas_modelos)
    metricas_df.to_csv(f"{base_output}/Tables/metricas_modelos_por_cenario.csv", index=False)

    # Importância das variáveis (Random Forest)
    importances_rf = pd.Series(model_rf.feature_importances_, index=[legendas_dict.get(v.replace('_bin', ''), v) for v in X.columns])
    fig, ax = plt.subplots(figsize=(12, 6))
    importances_rf.sort_values().plot(kind='barh', ax=ax)
    ax.set_title(f"Importância das Variáveis - Cenário {nome_cenario.capitalize()} (RF)")
    plt.xlim(0, .7)
    plt.tight_layout()
    plt.savefig(f"{base_output}/Imagens/Metricas/importancia_variaveis_rf_{nome_cenario}.png")
    plt.close()

    # Importância das variáveis (Logistic Regression via coeficientes absolutos)
    coef = model_lr.coef_[0]
    importances_lr = pd.Series(np.abs(coef), index=[legendas_dict.get(v.replace('_bin', ''), v) for v in X.columns])

    fig, ax = plt.subplots(figsize=(12, 6))
    importances_lr.sort_values().plot(kind='barh', ax=ax)
    ax.set_title(f"Importância das Variáveis - Cenário {nome_cenario.capitalize()} (LR)")
    plt.xlim(0, 3)
    plt.tight_layout()
    plt.savefig(f"{base_output}/Imagens/Metricas/importancia_variaveis_lr_{nome_cenario}.png")
    plt.close()


=== CENÁRIO: CONSERVADOR ===
Random Forest
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1116
           1       0.54      0.80      0.64       349

    accuracy                           0.79      1465
   macro avg       0.73      0.79      0.75      1465
weighted avg       0.83      0.79      0.80      1465

Matriz de Confusão:  [[877 239]
 [ 70 279]]
Logistic Regression
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1116
           1       0.58      0.74      0.65       349

    accuracy                           0.81      1465
   macro avg       0.75      0.79      0.76      1465
weighted avg       0.83      0.81      0.82      1465

Matriz de Confusão:  [[928 188]
 [ 89 260]]
Modelos salvos: ../Output/Analise_03/Modelos/model_randomforest_conservador.pkl e ../Output/Analise_03/Modelos/model_logisticregression_conservador.pkl

=== CENÁRIO: MODERADO ===
Random Forest
 

##### Analise dos Modelos

In [93]:
nomes_modelos_legiveis = {
    'RandomForest': 'Random Forest',
    'LogisticRegression': 'Regressão Logística'
}

df_metricas = pd.read_csv(f'{base_output}/Tables/metricas_modelos_por_cenario.csv')
df_metricas['modelo_legivel'] = df_metricas['modelo'].map(nomes_modelos_legiveis)

In [94]:
df_top_scores = (
    df_metricas.sort_values('value', ascending=False)
    .groupby(['classe', 'metric'])
    .first()
    .reset_index()
)

df_top_scores = df_top_scores[['classe', 'metric', 'cenario', 'modelo', 'value']]
df_top_scores

,classe,metric,cenario,modelo,value
0,0,f1-score,arrojado,RandomForest,0.882847
1,0,precision,conservador,RandomForest,0.926082
2,0,recall,arrojado,LogisticRegression,0.989247
3,1,f1-score,moderado,LogisticRegression,0.666667
4,1,precision,arrojado,LogisticRegression,0.850000
5,1,recall,conservador,RandomForest,0.799427


In [95]:
def plot_metricas_modelos(df, titulo, output_path, filtro_classe=None, col=None, row=None, legenda_fora=True):
    if filtro_classe is not None:
        df = df[df['classe'] == filtro_classe]

    modelos_legiveis = {
        'RandomForest': 'Random Forest',
        'LogisticRegression': 'Regressão Logística'
    }
    df = df.copy()
    df['modelo_legivel'] = df['modelo'].map(modelos_legiveis)

    g = sns.catplot(
        data=df,
        x='metric',
        y='value',
        hue='modelo_legivel',
        col=col,
        row=row,
        kind='bar',
        palette='Set2',
        height=4,
        aspect=1.1,
        sharey=True
    )

    g.set_titles('Cenário: {col_name}' if col else '')
    if row:
        g.set_titles('Cenário: {col_name} | Classe: {row_name}')

    g.set_axis_labels('Métrica', 'Desempenho do Modelo')
    g.set(ylim=(0, 1))

    if g._legend:
        g._legend.set_title(None)
        if legenda_fora:
            g._legend.set_bbox_to_anchor((.86, 0.5))
            g._legend.set_loc('center left')

    for ax in g.axes.flat:
        for label in ax.get_xticklabels():
            label.set_rotation(0)

    plt.suptitle(titulo, y=1.03, fontsize=14)
    g.tight_layout()
    g.savefig(output_path)
    plt.close()


In [92]:
# Carrega e prepara o DataFrame
df_metricas = pd.read_csv(f'{base_output}/Tables/metricas_modelos_por_cenario.csv')

# 1. Comparação geral (sem filtro)
plot_metricas_modelos(
    df=df_metricas,
    titulo='Desempenho Geral dos Modelos - Previsão de Paradas e Não-Paradas',
    output_path=f'{base_output}/Imagens/Modelos/comparacao_geral_modelos.png'
)

# 2. Comparação por cenário - Classe: Parada
plot_metricas_modelos(
    df=df_metricas,
    titulo='Comparação por Cenário - Classe: Parada',
    filtro_classe=1,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/comparacao_por_cenario_parada.png'
)

# 3. Comparação por cenário - Classe: Não-Parada
plot_metricas_modelos(
    df=df_metricas,
    titulo='Comparação por Cenário - Classe: Não-Parada',
    filtro_classe=0,
    col='cenario',
    output_path=f'{base_output}/Imagens/Modelos/comparacao_por_cenario_nao_parada.png'
)

# 4. Comparação por cenário e por classe
plot_metricas_modelos(
    df=df_metricas,
    titulo='Comparação por Cenário e Classe',
    col='cenario',
    row='classe',
    output_path=f'{base_output}/Imagens/Modelos/comparacao_por_cenario_e_classe.png'
)